In [1]:
#instead of one-hot encoding, use 1,-1 on single node
import pandas as pd

df = pd.read_csv('raw/삼성전자.csv')

In [2]:
import csv

row_len = 30 # each row has 30 items, the last item being the buy/sell signal
iter_len = len(df['Close'])-row_len
start_q = list(df['Close'][:row_len])

filename = 'prep1/삼성전자.csv'

# Labeling: increase = 1, decrease = -1

with open(filename, 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow(['Day '+str(i) for i in range(1,row_len-1)]+['Buy or Sell'])
    for i in range(iter_len):
        temp = start_q.copy()
        if temp[-1]>temp[-2]:
            label = 1
        else: label = -1
        for j in range(len(temp)-2):
            temp[j]=temp[j+1]/temp[j]
        temp[-2]=label
        temp.pop()
        wt.writerow(temp)
        del start_q[0]
        start_q.append(df['Close'][i+row_len])
        

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


f = pd.read_csv(filename)
labels = f.axes[1]
rawX = f.drop(columns=labels[-1], axis=1).values
rawY = f.drop(columns=labels[:-1], axis=1).values

In [4]:
newY = []
empty = np.zeros((1,), dtype=int)

for val in rawY:
    row = empty.copy()
    row[0]=val
    newY.append(row)

x=np.array(rawX)
y=np.array(newY)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)
    

In [5]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, H)
        self.h1 = nn.Linear(H, 2*H)
        self.h2 = nn.Linear(2*H, 4*H)
        self.l2 = nn.Linear(4*H, D_out)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        y = F.relu(self.l2(x))
        return y


D_in, D_out = 28, 1 #input, output 

def run(H, lr, epochs):
  
    #training
    
    model = Net(D_in, H, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation
    total=0
    correct=0
    
    for idx, inp in enumerate(x):
        total+=1
        y_pred = model(inp.float()).cpu().detach().numpy()
        y_truth = list(newY[idx])
        print(y_pred, y_truth)
        if y_pred == y_truth: correct+=1
    
    print(round(100*correct/total,2))


In [6]:
accs = []
accs.append(run(56, 1e-3, 1000))
print(accs[0])

0 5448.0
1 5448.0
2 5448.0
3 5448.0
4 5448.0
5 5448.0
6 5448.0
7 5448.0
8 5448.0
9 5448.0
10 5448.0
11 5448.0
12 5448.0
13 5448.0
14 5448.0
15 5448.0
16 5448.0
17 5448.0
18 5448.0
19 5448.0
20 5448.0
21 5448.0
22 5448.0
23 5448.0
24 5448.0
25 5448.0
26 5448.0
27 5448.0
28 5448.0
29 5448.0
30 5448.0
31 5448.0
32 5448.0
33 5448.0
34 5448.0
35 5448.0
36 5448.0
37 5448.0
38 5448.0
39 5448.0
40 5448.0
41 5448.0
42 5448.0
43 5448.0
44 5448.0
45 5448.0
46 5448.0
47 5448.0
48 5448.0
49 5448.0
50 5448.0
51 5448.0
52 5448.0
53 5448.0
54 5448.0
55 5448.0
56 5448.0
57 5448.0
58 5448.0
59 5448.0
60 5448.0
61 5448.0
62 5448.0
63 5448.0
64 5448.0
65 5448.0
66 5448.0
67 5448.0
68 5448.0
69 5448.0
70 5448.0
71 5448.0
72 5448.0
73 5448.0
74 5448.0
75 5448.0
76 5448.0
77 5448.0
78 5448.0
79 5448.0
80 5448.0
81 5448.0
82 5448.0
83 5448.0
84 5448.0
85 5448.0
86 5448.0
87 5448.0
88 5448.0
89 5448.0
90 5448.0
91 5448.0
92 5448.0
93 5448.0
94 5448.0
95 5448.0
96 5448.0
97 5448.0
98 5448.0
99 5448.0
100 5448.0

762 5448.0
763 5448.0
764 5448.0
765 5448.0
766 5448.0
767 5448.0
768 5448.0
769 5448.0
770 5448.0
771 5448.0
772 5448.0
773 5448.0
774 5448.0
775 5448.0
776 5448.0
777 5448.0
778 5448.0
779 5448.0
780 5448.0
781 5448.0
782 5448.0
783 5448.0
784 5448.0
785 5448.0
786 5448.0
787 5448.0
788 5448.0
789 5448.0
790 5448.0
791 5448.0
792 5448.0
793 5448.0
794 5448.0
795 5448.0
796 5448.0
797 5448.0
798 5448.0
799 5448.0
800 5448.0
801 5448.0
802 5448.0
803 5448.0
804 5448.0
805 5448.0
806 5448.0
807 5448.0
808 5448.0
809 5448.0
810 5448.0
811 5448.0
812 5448.0
813 5448.0
814 5448.0
815 5448.0
816 5448.0
817 5448.0
818 5448.0
819 5448.0
820 5448.0
821 5448.0
822 5448.0
823 5448.0
824 5448.0
825 5448.0
826 5448.0
827 5448.0
828 5448.0
829 5448.0
830 5448.0
831 5448.0
832 5448.0
833 5448.0
834 5448.0
835 5448.0
836 5448.0
837 5448.0
838 5448.0
839 5448.0
840 5448.0
841 5448.0
842 5448.0
843 5448.0
844 5448.0
845 5448.0
846 5448.0
847 5448.0
848 5448.0
849 5448.0
850 5448.0
851 5448.0
852 5448.0

[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]


[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]


[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [

[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.]

[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [1]
[0.] [-1]
[0.] [-1]
[0.] [1]
[0.] [1]
[0